# Titanic - Machine Learning from Disaster

- Vamos utilizar os [dados disponíveis no Kaggle](https://www.kaggle.com/competitions/titanic)
    - É um dataset de **competição**
    - O resultado é avaliado através da **acurácia**:
        - _"Sua pontuação é a porcentagem de passageiros que você prevê corretamente. Isso é conhecido como acurácia."_

In [430]:
import pandas as pd

In [431]:
treino = pd.read_csv('train.csv')

In [432]:
teste = pd.read_csv('test.csv')

> Como já houve uma tentativa para esse desafio, vamos apenas replicar algumas etapas e tentar melhorar a precisão de nossa previsão.

In [433]:
# Eliminando as colunas com elevada cardinalidade
treino = treino.drop(['Name','Ticket','Cabin'],axis=1)
teste = teste.drop(['Name','Ticket','Cabin'],axis=1)

In [434]:
# Usando a mediana para imputar os valores nulos na coluna de idade
treino.loc[treino.Age.isnull(),'Age'] = treino.Age.median()
teste.loc[teste.Age.isnull(),'Age'] = teste.Age.median()

In [435]:
# Tratando a coluna Embarked da base de treino usando a moda 
treino.loc[treino.Embarked.isnull(),'Embarked'] = treino.Embarked.mode()[0]

In [436]:
# E também a coluna Fare da base de teste usando a média
teste.loc[teste.Fare.isnull(),'Fare'] = teste.Fare.mean()

In [437]:
treino.dtypes=='object'

PassengerId    False
Survived       False
Pclass         False
Sex             True
Age            False
SibSp          False
Parch          False
Fare           False
Embarked        True
dtype: bool

In [438]:
treino.columns[treino.dtypes=='object']

Index(['Sex', 'Embarked'], dtype='object')

In [439]:
treino.Sex.value_counts()

Sex
male      577
female    314
Name: count, dtype: int64

In [440]:
treino.Embarked.value_counts()

Embarked
S    646
C    168
Q     77
Name: count, dtype: int64

### Variável Sex

Vamos transformar a coluna Sex código númerico para que a coluna possa ser usada no modelo de aprendizagem

In [441]:
sex = {'male': 1,
       'female': 0}

In [442]:
treino['MaleCheck'] = treino.Sex.apply(lambda x: 1 if x == 'male' else 0)

In [443]:
treino[['Sex', 'MaleCheck']].value_counts()

Sex     MaleCheck
male    1            577
female  0            314
Name: count, dtype: int64

In [444]:
teste['MaleCheck'] = teste.Sex.apply(lambda x: 1 if x == 'male' else 0)

In [445]:
teste[['Sex', 'MaleCheck']].value_counts()

Sex     MaleCheck
male    1            266
female  0            152
Name: count, dtype: int64

### OneHotEncoder na coluna Embarked: Por quê

- OneHotEncoder é uma técnica de pré-processamento usada em aprendizado de máquina para lidar com variáveis categóricas. Ele converte variáveis categóricas em uma representação binária, criando uma nova coluna para cada categoria com valores binários indicando a presença ou ausência dessa categoria.

- Essa técnica é especialmente interessante por que há um pequeno número de categorias (3) na coluna 'Embarked', melhorando a eficiencia de aprendizado de máquina, Permitindo que o modelo compreenda e utilize efetivamente informações de variáveis categóricas em seus cálculos.

    - Ao criar colunas binárias para cada local de embarque, o modelo pode interpretar de maneira mais eficaz as informações categóricas, melhorando a capacidade preditiva do algoritmo. Essa abordagem é especialmente relevante quando lidamos com um conjunto de dados contendo um número restrito de categorias, contribuindo para a interpretação e desempenho aprimorados do modelo.

In [446]:
from sklearn.preprocessing import OneHotEncoder

In [447]:
# Criando o encoder
ohe = OneHotEncoder(handle_unknown='ignore',dtype='int32')

In [448]:
ohe = ohe.fit(treino[['Embarked']])

In [449]:
# Fazendo a transformação
ohe.transform(treino[['Embarked']]).toarray()

array([[0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       ...,
       [0, 0, 1],
       [1, 0, 0],
       [0, 1, 0]])

In [450]:
# Transformando esse resultado em um DataFrame
ohe_df = pd.DataFrame(ohe.transform(treino[['Embarked']]).toarray(),columns=ohe.get_feature_names_out())
ohe_df.head(3)

,Embarked_C,Embarked_Q,Embarked_S
0,0,0,1
1,1,0,0
2,0,0,1


In [451]:
# Concatenando o dataframe 
treino = pd.concat([treino,ohe_df],axis=1)

In [452]:
# Verificando os valores 
treino[['Embarked','Embarked_C','Embarked_Q','Embarked_S']].value_counts()

Embarked  Embarked_C  Embarked_Q  Embarked_S
S         0           0           1             646
C         1           0           0             168
Q         0           1           0              77
Name: count, dtype: int64

- Agora fazendo o mesmo para a base de teste usando o encoder ohe que criamos acima

In [453]:
ohe_df = pd.DataFrame(ohe.transform(teste[['Embarked']]).toarray(),columns=ohe.get_feature_names_out())

In [454]:
teste = pd.concat([teste,ohe_df],axis=1)

In [455]:
# Verificando os valores
teste[['Embarked','Embarked_C','Embarked_Q','Embarked_S']].value_counts()

Embarked  Embarked_C  Embarked_Q  Embarked_S
S         0           0           1             270
C         1           0           0             102
Q         0           1           0              46
Name: count, dtype: int64

In [456]:
treino.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,MaleCheck,Embarked_C,Embarked_Q,Embarked_S
0,1,0,3,male,22.0,1,0,7.2500,S,1,0,0,1
1,2,1,1,female,38.0,1,0,71.2833,C,0,1,0,0
2,3,1,3,female,26.0,0,0,7.9250,S,0,0,0,1
3,4,1,1,female,35.0,1,0,53.1000,S,0,0,0,1
4,5,0,3,male,35.0,0,0,8.0500,S,1,0,0,1


In [457]:
# Eliminando as colunas que foram tratadas/modificadas
treino = treino.drop(['Sex','Embarked'],axis=1)
teste = teste.drop(['Sex','Embarked'],axis=1)

### Utilizando outros modelos para fazer a previsão
- Foram selecionados algoritmos diferentes dos que testamos nas tentativas anteriores considerando os [outros algoritmos disponíveis no scikit-learn](https://scikit-learn.org/stable/supervised_learning.html#supervised-learning)

- Para essa nova tentativa, deixamos o modelo que conseguiu a melhor acurária para realizar a comparação com outros novos modelos
    - **Regressão Logística**
        - https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression
    - **Random Forest**
        - https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier
    - **MLPClassifier (Redes Neurais)**
        - https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html#sklearn.neural_network.MLPClassifier
        

- train_test_split é uma função da biblioteca Scikit-Learn que tem o objetivo de dividir um conjunto de dados em duas partes: uma para treinamento do modelo e outra para avaliação do desempenho do modelo. A função garante que o modelo generalize bem para novos dados não vistos, prevenindo overfitting. O conjunto de treinamento é usado para treinar o modelo, enquanto o conjunto de teste é reservado para avaliação de desempenho.

In [458]:
from sklearn.model_selection import train_test_split

In [459]:
X = treino.drop(['PassengerId','Survived'],axis=1)
# A tabela abaixo atribui o resultado que esperamos, para verificar se há um modelo que possa prever os sobreviventes.
y = treino.Survived

In [460]:
# Separando em treino e validação
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.40)

### Criando o modelo de teste com **Random Forest Classifier**

In [461]:
from sklearn.ensemble import RandomForestClassifier

In [462]:
# Criando o classificador
# clf -> classificação
# rf -> Random Forest  Classifier
clf_rf = RandomForestClassifier()

In [463]:
# Fazendo o fit com os dados (treinando o modelo)
clf_rf = clf_rf.fit(X_train,y_train)

In [464]:
# Previsão
y_pred_rf = clf_rf.predict(X_val)

### Para o **MLP classifier**

In [465]:
from sklearn.neural_network import MLPClassifier

In [466]:
# Criando o classificador
# clf -> classificação
# mlp -> modelo usado (MLP classifier)
clf_mlp = MLPClassifier(max_iter=8000)

In [467]:
# Fazendo o fit com os dados (treinando o modelo)
clf_mlp = clf_mlp.fit(X_train,y_train)

In [468]:
y_pred_mlp = clf_mlp.predict(X_val)

### Para a **Regressão Logística**

In [469]:
from sklearn.linear_model import LogisticRegression

In [470]:
# Criando o classificador
# clf -> classificador
# rl -> Regressão Logística
clf_rl = LogisticRegression()

In [471]:
# Fazendo o fit com os dados (treinando o modelo)
clf_rl = clf_rl.fit(X_train,y_train)

c:\Users\Lucas\OneDrive\Documentos\avancandoPython\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [472]:
y_pred_rl = clf_rl.predict(X_val)

### Para finalizar, vamos avaliar esses modelos treinados:
- Para fazer essa análise, será usado:
    - Acurácia (método de avaliação usado na competição):
        - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html
    - Matriz de confusão (ajuda a visualizar a distribuição dos erros):
        - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html

In [473]:
from sklearn.metrics import accuracy_score

In [474]:
as_rf = accuracy_score(y_val, y_pred_rf) # Verificando acurácia para a árvore de classificação
as_mlp = accuracy_score(y_val, y_pred_mlp) # Verificando acurácia para o knn
as_rl = accuracy_score(y_val, y_pred_rl) # Para a regressão logística

print(as_rf, as_mlp, as_rl)

0.8123249299719888 0.8067226890756303 0.7871148459383753


- Salvando esses valores em um DataFrame para visualizarmos a evolução dos testes

In [475]:
# Criando primeiramente o DataFrame
as_df = pd.DataFrame({
    'modelos': ['random forest.','MLP','reg. log.'],
    'result1': [as_rf,as_mlp,as_rl] 
})

as_df

,modelos,result1
0,random forest.,0.812325
1,MLP,0.806723
2,reg. log.,0.787115


### Avaliando a **matriz de confusão**

- Utilizaremos a matriz de confusão afim de validar mais uma vez a ferramenta mais eficiente aplicada ao nosso teste. 
    - O módulo confusion_matrix do Scikit-Learn é utilizado para avaliar o desempenho de modelos de classificação. Ele cria uma tabela que mostra a contagem de acertos e erros do modelo em diferentes categorias

In [476]:
from sklearn.metrics import confusion_matrix

In [477]:
# Para a random forest
confusion_matrix(y_val, y_pred_rf)

array([[205,  21],
       [ 46,  85]], dtype=int64)

In [478]:
# Para o MLP (redes neurais)
confusion_matrix(y_val, y_pred_mlp)

array([[202,  24],
       [ 45,  86]], dtype=int64)

In [479]:
# Para a regressão logística
confusion_matrix(y_val, y_pred_rl)

array([[197,  29],
       [ 47,  84]], dtype=int64)

### Fazendo a previsão para os dados de teste
- Vamos usar o modelo com melhor precisão para fazer o predict na base de teste

In [480]:
X_train.head(3)

,Pclass,Age,SibSp,Parch,Fare,MaleCheck,Embarked_C,Embarked_Q,Embarked_S
518,2,36.0,1,0,26.0000,0,0,0,1
72,2,21.0,0,0,73.5000,1,0,0,1
294,3,24.0,0,0,7.8958,1,0,0,1


In [481]:
teste.head(3)

,PassengerId,Pclass,Age,SibSp,Parch,Fare,MaleCheck,Embarked_C,Embarked_Q,Embarked_S
0,892,3,34.5,0,0,7.8292,1,0,1,0
1,893,3,47.0,1,0,7.0000,0,0,0,1
2,894,2,62.0,0,0,9.6875,1,0,1,0


In [482]:
X_teste = teste.drop('PassengerId',axis=1)

In [487]:
# Utilizando o melhor modelo na base de teste
y_pred = clf_rf.predict(X_teste)

In [488]:
teste['Survived'] = y_pred

In [489]:
base_envio = teste[['PassengerId','Survived']]

In [490]:
base_envio.to_csv('result_random_forest.csv',index=False)